In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import numpy as np
import scipy as sp
import scipy.interpolate
import pandas as pd
import rwthcolors
import eurographics_style
import glob
import re

In [ ]:
output_dir = '/local/born/layout-embedding-output'
optimization_pipeline_output_dir = output_dir + '/optimization_timeline'

In [ ]:
ub_fns = glob.glob(optimization_pipeline_output_dir + '/*_upper_bound.csv')

runs = []

for fn in ub_fns:
    m = re.search('(\d+)_upper_bound\.csv', fn)
    seed = int(m.group(1))
    data = pd.read_csv(fn)
    num_events = len(data)
    run = {'seed': seed, 'num_events': num_events}
    runs.append(run)

runs = sorted(runs, key=lambda r: r['num_events'], reverse=True)
for run in runs:
    seed = run['seed']
    num_events = run['num_events']
    print(f'Seed {seed}: {num_events} upper bound events')

In [ ]:
best_seed = runs[0]['seed']

lb_fn = optimization_pipeline_output_dir + f'/{best_seed}_lower_bound.csv'
ub_fn = optimization_pipeline_output_dir + f'/{best_seed}_upper_bound.csv'

lb_data = pd.read_csv(lb_fn)
ub_data = pd.read_csv(ub_fn)

In [ ]:
def plot_run(lb_data, ub_data):
    min_t = min(lb_data['t'].min(), ub_data['t'].min())
    max_t = max(lb_data['t'].max(), ub_data['t'].max())

    t_resampled = np.sort(np.concatenate((lb_data['t'], ub_data['t'])))

    lb_interp = sp.interpolate.interp1d(lb_data['t'], lb_data['lower_bound'], kind='previous', fill_value='extrapolate')
    lb_resampled = lb_interp(t_resampled)

    ub_interp = sp.interpolate.interp1d(ub_data['t'], ub_data['upper_bound'], kind='previous', fill_value='extrapolate')
    ub_resampled = ub_interp(t_resampled)
    
    plt.figure()
    plt.xlabel('Time')

    plt.fill_between(t_resampled, lb_resampled, ub_resampled, color='rwth:blue10', step='post')

    plt.step(t_resampled, ub_resampled, label='Upper Bound', c='rwth:blue', where='post')
    plt.step(t_resampled, lb_resampled, label='Lower Bound', c='rwth:blue50', where='post')

    point_size = 10.0
    plt.scatter(lb_data['t'], lb_data['lower_bound'], c='rwth:blue50', s=point_size)
    plt.scatter(ub_data['t'], ub_data['upper_bound'], c='rwth:blue', s=point_size)

    plt.legend()

In [ ]:
plot_run(lb_data, ub_data)

In [ ]:
seed = 0
lb_fn = optimization_pipeline_output_dir + f'/{seed}_lower_bound.csv'
ub_fn = optimization_pipeline_output_dir + f'/{seed}_upper_bound.csv'
lb_data = pd.read_csv(lb_fn)
ub_data = pd.read_csv(ub_fn)
plot_run(lb_data, ub_data)